In [1]:
import sys
import torch
sys.path.append('/cluster/tufts/hugheslab/swilli26/SNGP-IMPLEMENTATION/src')
from sklearn import metrics
import numpy as np
import math

import losses
from models import StandardThreeLayerDNN, SNGPThreeLayerDNN, RandomFeatureGaussianProcess, ResFFN12_128, SNGP_ResFFN12_128
from utils import train_model, create_moons_data_loaders, get_data_from_loader, apply_bounded_spectral_norm, compute_covariance
from viz_utils import display_data_splits, plot_loss, plot_preds, plot_probas_over_dense_grid, plot_probabilities, plot_probabilities_gp, plot_thresholded_predictions
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


# **SNGP TOY EXAMPLE:**

In [2]:
train_loader, val_loader, test_loader = create_moons_data_loaders(3000, 3000)
display_data_splits(train_loader, val_loader, test_loader)

In [3]:
input_size = 2
output_size = 2
hidden_size = 64

model = StandardThreeLayerDNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size)
# apply_bounded_spectral_norm(model, name='weight', spec_norm_iteration=15, spec_norm_bound=1.0)
# summary(model, input_size=(1, 2))

model, info = train_model(model, 'cuda', train_loader, val_loader, losses, n_epochs=100, lr=0.1, do_early_stopping=True, model_filename="/cluster/tufts/hugheslab/swilli26/SNGP-IMPLEMENTATION/results/models/best_model.pth")

100%|██████████| 101/101 [00:03<00:00, 29.54it/s, tr_xent=0.257, tr_err=0.075, va_xent=0.287, va_err=0.0883, batch_done=1]

Finished after epoch 100, best epoch=100


In [8]:
%reload_ext autoreload
plot_probabilities(model, test_loader, device='cuda')

torch.Size([160000, 2])


In [12]:
%reload_ext autoreload
from models import RandomFeatureGaussianProcess

input_size = 2
output_size = 2

sngp_model = RandomFeatureGaussianProcess(in_features=input_size, out_features=output_size, rank=1_024, lengthscale=0.1, outputscale=0.5)

In [13]:
sngp_model, new_info = train_model(sngp_model, 'cuda', train_loader, val_loader, losses, n_epochs=1_000, lr=2.0, do_early_stopping=True)
plot_loss(new_info)

100%|██████████| 1001/1001 [00:24<00:00, 41.39it/s, tr_xent=0.072, tr_err=0.00111, va_xent=0.0842, va_err=0.00167, batch_done=1]


Finished after epoch 1000, best epoch=1000


In [14]:
%reload_ext autoreload
plot_probabilities_gp(sngp_model, train_loader, num_samples=10_000, compute_covariance=True, device='cuda')

AttributeError: 'RandomFeatureGaussianProcess' object has no attribute 'update_covariance_from_loader'

In [8]:
%reload_ext autoreload
preds_file = "preds.pt"
plot_thresholded_predictions(preds_file, test_loader, threshold=0.5)

In [9]:
%reload_ext autoreload
preds_file = "preds.pt"
plot_thresholded_predictions(preds_file, test_loader, threshold=0.6)

In [10]:
%reload_ext autoreload
preds_file = "preds.pt"
plot_thresholded_predictions(preds_file, test_loader, threshold=0.65)

In [11]:
%reload_ext autoreload
preds_file = "preds.pt"
plot_thresholded_predictions(preds_file, test_loader, threshold=0.70)